In [47]:
!pip install torch torchquantum

Killed


In [4]:
s

NameError: name 's' is not defined

In [3]:
!pip install pennylane

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached future-0.18.2-py3-none-any.whl
You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [17]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

ModuleNotFoundError: No module named 'torch'

In [ ]:
from datasets import TorchDataset
from create_gate_circs import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_data = TorchDataset(dataset, 'angle', 1, True)
test_data = TorchDataset(dataset, 'angle', 1, False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1024, sampler=torch.utils.data.RandomSampler(train_data))

val_test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=100, sampler=torch.utils.data.SequentialSampler(test_data))

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

for i in range(300, 333):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)

    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        accs_history = train_tq_model(model, opt, loss, train_data_loader, test_data_loader, 120000, 1000, 1000)
        
#         curr_val_losses = []
#         curr_acc = 0
        
#         for k in range(100):
#             val_x, val_y = next(iter(val_test_data_loader))

#             val_preds = model(val_x)
#             curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
#             val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()

#             curr_acc += np.mean(val_class == val_y.detach().numpy().flatten())
        
#         np.save(curr_train_dir + '/accs_history_{}.npy'.format(j + 1), accs_history)
        
#         losses_list.append(curr_val_losses)
#         accs_list.append(curr_acc / 100)
        
#         print(curr_acc)
        
    torch.save(model.state_dict(), curr_dir + '/model.pt')
#     np.save(curr_dir + '/val_losses.npy', losses_list)
#     np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Accuracy: 0.0478515625
Step 1001 | Accuracy: 0.1650390625
Step 2001 | Accuracy: 0.212890625
Step 3001 | Accuracy: 0.2724609375
Step 4001 | Accuracy: 0.287109375
Step 5001 | Accuracy: 0.25
Step 6001 | Accuracy: 0.27734375
Step 7001 | Accuracy: 0.259765625
Step 8001 | Accuracy: 0.2744140625
Step 9001 | Accuracy: 0.294921875
Step 10001 | Accuracy: 0.2861328125
Step 11001 | Accuracy: 0.294921875
Step 12001 | Accuracy: 0.2724609375
Step 13001 | Accuracy: 0.2900390625
Step 14001 | Accuracy: 0.2900390625
Step 15001 | Accuracy: 0.2841796875
Step 16001 | Accuracy: 0.3115234375
Step 17001 | Accuracy: 0.314453125
Step 18001 | Accuracy: 0.3212890625
Step 19001 | Accuracy: 0.30859375
Step 20001 | Accuracy: 0.3193359375
Step 21001 | Accuracy: 0.3046875
Step 22001 | Accuracy: 0.3173828125
Step 23001 | Accuracy: 0.326171875
Step 24001 | Accuracy: 0.3359375
Step 25001 | Accuracy: 0.3134765625
Step 26001 | Accuracy: 0.3095703125
Step 27001 | Accuracy: 0.3154296875
Step 28001 | Accuracy: 0.30761

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibm_nairobi'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(200, 400):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-10-21 17:38:15,843: Credentials are already in use. The existing account in the session will be replaced.


## fix bank errors

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_vec_loss

import pennylane as qml
import numpy as np
import os

dataset = 'bank'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 8
num_params = 24

err_inds = np.array([[for j in range(5)] for i in range(8)]).flatten()

for i in range(875, 1000):
    curr_dir = './experiment_data/mnist_4/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0, 1], 'exp')    

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        
        info = train_qnn(circ, x_train, y_train, x_test, y_test, [num_params], 16000, 0.05, 1, mse_vec_loss, verbosity=127300, 
                                                                                        loss_window=50, init_params=None, 
                                                                                        acc_thres=1.1, shuffle=True, print_loss=50)
        
        val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
        val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
        
        np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
        np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])
        
        losses_list.append(val_loss)
        accs_list.append(acc)
        
    np.savetxt(curr_dir + '/val_losses.txt', losses_list)
    np.savetxt(curr_dir + '/accs.txt', accs_list)

## supernet

In [6]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_batched_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4

device_name = 'ibm_nairobi'
# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [8, 12]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 

        
        circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0], 'exp')

        val_exp_list = []
        
        for j in range(np.ceil(len(x_test) / 32).astype('int32')):
            curr_batch_params = np.concatenate([[curr_params] for k in range(min(32, len(x_test) - 32 * j))], 0)
            
            val_exp_list.append(circ(x_test[32 * j:min(len(x_test), 32* j + 32)], curr_batch_params))
            
        val_exps = np.concatenate(val_exp_list)
        val_losses = np.power(val_exps - y_test, 2)
        val_loss = np.mean(val_losses)
        acc = np.mean(val_losses < 1)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)    

ibmqfactory.load_account:WARNING:2022-10-07 17:50:00,677: Credentials are already in use. The existing account in the session will be replaced.


In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4
meas_qubits = [0]
num_steps = 8000

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [20]
search_nums = [100]
num_trials = [25]

device_name = 'ibmq_lima'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    circ_accs = np.genfromtxt(curr_params_dir + '/{}/searched_circ_accs.txt'.format(device_name))
    circ_losses = np.genfromtxt(curr_params_dir + '/{}/searched_circ_losses.txt'.format(device_name))
    circ_layers = np.genfromtxt(curr_params_dir + '/{}/searched_circ_layers.txt'.format(device_name)).astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/{}/search_{}'.format(device_name, search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(10, 25):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
            np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (193259.0410847615 > 2 * 90304.57161775761). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (193259.0410847615 > 2 * 90304.57161775761). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (193259.0410847615 > 2 * 90304.57161775761). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer

0.5675 1.0075315380096435 [284510 280432 278685 155525 214053]
[2022-10-13 14:59:51.625 tensorflow-2-3-cpu-py-ml-t3-medium-dbca98283d57d615662c4efa28c8:27 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-10-13 14:59:51.737 tensorflow-2-3-cpu-py-ml-t3-medium-dbca98283d57d615662c4efa28c8:27 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Step 1 | Sliding Loss Window : 1.1687044805071023
Step 51 | Sliding Loss Window : 1.1402462199108623
Step 101 | Sliding Loss Window : 1.0437048649701115
Step 151 | Sliding Loss Window : 1.0396026566029086
Step 201 | Sliding Loss Window : 1.062623836958626
Step 251 | Sliding Loss Window : 1.0298366369545082
Step 301 | Sliding Loss Window : 1.0179686008929167
Step 351 | Sliding Loss Window : 1.0607351176513753
Step 401 | Sliding Loss Window : 1.0554055148564374
Step 451 | Sliding Loss Window : 1.024237281303757
Step 501 | Sliding Loss Window : 1.0067491621465872
Step 55